In [62]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import xgboost as xgb
import operator


In [63]:
sample_sub_path = "../data/sample_submission.csv"
store_path = "../data/store.csv"
test_path = "../data/test.csv"
train_path = "../data/train.csv"

In [64]:
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}

In [65]:
sample_sub = pd.read_csv(sample_sub_path)
store = pd.read_csv(store_path)
test = pd.read_csv(test_path, parse_dates=[3])
train = pd.read_csv(train_path, parse_dates=[2])

In [66]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct"
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct"
3,4,c,c,620,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN


In [67]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1,1,0,0
1,2,3,4,2015-09-17,1,1,0,0
2,3,7,4,2015-09-17,1,1,0,0
3,4,8,4,2015-09-17,1,1,0,0
4,5,9,4,2015-09-17,1,1,0,0


In [68]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [69]:
train.loc[train.Sales == 0].shape

(172871, 9)

In [70]:
train.shape

(1017209, 9)

In [71]:
train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [72]:
test.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [73]:
pd.__version__

'0.13.1'

In [74]:
test.shape

(41088, 8)

In [75]:
test.loc[ test.Open.isnull(), 'Open' ] = 1
test.shape

(41088, 8)

In [76]:
test.isnull().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [77]:
test = pd.read_csv(test_path)
test.Open.isnull()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
...
41073    False
41074    False
41075    False
41076    False
41077    False
41078    False
41079    False
41080    False
41081    False
41082    False
41083    False
41084    False
41085    False
41086    False
41087    False
Name: Open, Length: 41088, dtype: bool

In [78]:
columns = ['Store', 'DayOfWeek', 'Promo']
medians = train.groupby( columns )['Sales']

In [79]:
medians

In [80]:
len(train.Date.unique())

942

In [81]:
train.Date.unique()

array(['2015-07-30T17:00:00.000000000-0700',
       '2015-07-29T17:00:00.000000000-0700',
       '2015-07-28T17:00:00.000000000-0700',
       '2015-07-27T17:00:00.000000000-0700',
       '2015-07-26T17:00:00.000000000-0700',
       '2015-07-25T17:00:00.000000000-0700',
       '2015-07-24T17:00:00.000000000-0700',
       '2015-07-23T17:00:00.000000000-0700',
       '2015-07-22T17:00:00.000000000-0700',
       '2015-07-21T17:00:00.000000000-0700',
       '2015-07-20T17:00:00.000000000-0700',
       '2015-07-19T17:00:00.000000000-0700',
       '2015-07-18T17:00:00.000000000-0700',
       '2015-07-17T17:00:00.000000000-0700',
       '2015-07-16T17:00:00.000000000-0700',
       '2015-07-15T17:00:00.000000000-0700',
       '2015-07-14T17:00:00.000000000-0700',
       '2015-07-13T17:00:00.000000000-0700',
       '2015-07-12T17:00:00.000000000-0700',
       '2015-07-11T17:00:00.000000000-0700',
       '2015-07-10T17:00:00.000000000-0700',
       '2015-07-09T17:00:00.000000000-0700',
       '20

In [82]:
len(test.Date.unique())

48

In [83]:
test.Date.unique()

array(['2015-09-17', '2015-09-16', '2015-09-15', '2015-09-14',
       '2015-09-13', '2015-09-12', '2015-09-11', '2015-09-10',
       '2015-09-09', '2015-09-08', '2015-09-07', '2015-09-06',
       '2015-09-05', '2015-09-04', '2015-09-03', '2015-09-02',
       '2015-09-01', '2015-08-31', '2015-08-30', '2015-08-29',
       '2015-08-28', '2015-08-27', '2015-08-26', '2015-08-25',
       '2015-08-24', '2015-08-23', '2015-08-22', '2015-08-21',
       '2015-08-20', '2015-08-19', '2015-08-18', '2015-08-17',
       '2015-08-16', '2015-08-15', '2015-08-14', '2015-08-13',
       '2015-08-12', '2015-08-11', '2015-08-10', '2015-08-09',
       '2015-08-08', '2015-08-07', '2015-08-06', '2015-08-05',
       '2015-08-04', '2015-08-03', '2015-08-02', '2015-08-01'], dtype=object)

In [84]:
train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000
mean,558.429727,3.998341,5773.818972,633.145946,0.830107,0.381515,0.178647
std,321.908651,1.997391,3849.926175,464.411734,0.375539,0.485759,0.383056
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,3727.000000,405.000000,1.000000,0.000000,0.000000
50%,558.000000,4.000000,5744.000000,609.000000,1.000000,0.000000,0.000000
75%,838.000000,6.000000,7856.000000,837.000000,1.000000,1.000000,0.000000
max,1115.000000,7.000000,41551.000000,7388.000000,1.000000,1.000000,1.000000


In [85]:
sample_sub.shape

(41088, 2)

In [86]:
sample_sub.head()

,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [87]:
sample_sub.describe()

,Id,Sales
count,41088.000000,41088
mean,20544.500000,0
std,11861.228267,0
min,1.000000,0
25%,10272.750000,0
50%,20544.500000,0
75%,30816.250000,0
max,41088.000000,0


In [88]:
test.shape

(41088, 8)

In [89]:
test.loc[ test.Open.isnull()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
479,480,622,4,2015-09-17,NaN,1,0,0
1335,1336,622,3,2015-09-16,NaN,1,0,0
2191,2192,622,2,2015-09-15,NaN,1,0,0
3047,3048,622,1,2015-09-14,NaN,1,0,0
4759,4760,622,6,2015-09-12,NaN,0,0,0
5615,5616,622,5,2015-09-11,NaN,0,0,0
6471,6472,622,4,2015-09-10,NaN,0,0,0
7327,7328,622,3,2015-09-09,NaN,0,0,0
8183,8184,622,2,2015-09-08,NaN,0,0,0
9039,9040,622,1,2015-09-07,NaN,0,0,0


In [90]:
pd.DataFrame(test.Store.unique()).isin(train.Store).count()

0    856
dtype: int64

In [91]:
test.Store.value_counts()

1087    48
107     48
43      48
11      48
1066    48
970     48
938     48
906     48
874     48
842     48
810     48
778     48
746     48
714     48
650     48
...
629    48
597    48
533    48
117    48
501    48
405    48
373    48
341    48
309    48
277    48
245    48
213    48
181    48
149    48
32     48
Length: 856, dtype: int64

In [92]:
store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [94]:
feature = []
feature.extend(['store', 'promo'])
print feature

['store', 'promo']


In [95]:
test = pd.read_csv(test_path, parse_dates=[3])
test = pd.merge(test, store, on = "Store", how = "left")
print test.head()
test.describe()


     Id  Store  DayOfWeek       Date  Open  Promo StateHoliday  SchoolHoliday  \
0     1      1          4 2015-09-17     1      1            0              0   
1   857      1          3 2015-09-16     1      1            0              0   
2  1713      1          2 2015-09-15     1      1            0              0   
3  2569      1          1 2015-09-14     1      1            0              0   
4  3425      1          7 2015-09-13     0      0            0              0   

  StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0         c          a                 1270                          9   
1         c          a                 1270                          9   
2         c          a                 1270                          9   
3         c          a                 1270                          9   
4         c          a                 1270                          9   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYea

,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000,40992.000000,25872.000000,25872.000000,41088.000000,23856.000000,23856.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487,5088.583138,7.035250,2008.641929,0.580607,24.426559,2011.820926
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802,7225.487467,3.143015,6.862400,0.493466,14.161312,1.692166
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000,720.000000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000,2425.000000,7.000000,2010.000000,1.000000,22.000000,2012.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000,6480.000000,9.000000,2012.000000,1.000000,37.000000,2013.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000,75860.000000,12.000000,2015.000000,1.000000,49.000000,2015.000000


In [97]:
def promo2_indicator(row):
    if row['PromoInterval'] is np.NaN:
        return 0
    elif monthdic[row['month']-1] in row['PromoInterval']:
        return 1
    else:
        return 0

test['Promo2Indicator'] = test.apply(promo2_indicator, axis = 1)
print test.head()

KeyError: ('month', u'occurred at index 48')

In [99]:
train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000
mean,558.429727,3.998341,5773.818972,633.145946,0.830107,0.381515,0.178647
std,321.908651,1.997391,3849.926175,464.411734,0.375539,0.485759,0.383056
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,3727.000000,405.000000,1.000000,0.000000,0.000000
50%,558.000000,4.000000,5744.000000,609.000000,1.000000,0.000000,0.000000
75%,838.000000,6.000000,7856.000000,837.000000,1.000000,1.000000,0.000000
max,1115.000000,7.000000,41551.000000,7388.000000,1.000000,1.000000,1.000000


In [100]:
train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [101]:
test.isnull().sum()

Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                            11
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment                       0
CompetitionDistance             96
CompetitionOpenSinceMonth    15216
CompetitionOpenSinceYear     15216
Promo2                           0
Promo2SinceWeek              17232
Promo2SinceYear              17232
PromoInterval                17232
dtype: int64

In [104]:
test.loc[test.Open.isnull(), 'Open'] =1

In [103]:
test.Open.isnull()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
...
41073    False
41074    False
41075    False
41076    False
41077    False
41078    False
41079    False
41080    False
41081    False
41082    False
41083    False
41084    False
41085    False
41086    False
41087    False
Name: Open, Length: 41088, dtype: bool

In [105]:
test.dtypes

Id                                    int64
Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Open                                float64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object

In [106]:
train.dtypes

Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

In [108]:
test['dayofweek'] = test.Date.apply(lambda x: x.dayofweek)
test.dtypes

Id                                    int64
Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Open                                float64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
day                                   int64
dayofweek                             int64
dtype: object

In [109]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,day,dayofweek
0,1,1,4,2015-09-17,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,17,3
1,857,1,3,2015-09-16,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,16,2
2,1713,1,2,2015-09-15,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,15,1
3,2569,1,1,2015-09-14,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,14,0
4,3425,1,7,2015-09-13,0,0,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,13,6


In [110]:
test.isnull().sum()

Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                             0
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment                       0
CompetitionDistance             96
CompetitionOpenSinceMonth    15216
CompetitionOpenSinceYear     15216
Promo2                           0
Promo2SinceWeek              17232
Promo2SinceYear              17232
PromoInterval                17232
day                              0
dayofweek                        0
dtype: int64

In [111]:
test.shape

(41088, 19)

In [113]:
17232 / 41088.0

0.41939252336448596

In [121]:
test.iloc[4,:]

Id                                          3425
Store                                          1
DayOfWeek                                      7
Date                         2015-09-13 00:00:00
Open                                           0
Promo                                          0
StateHoliday                                   0
SchoolHoliday                                  0
StoreType                                      c
Assortment                                     a
CompetitionDistance                         1270
CompetitionOpenSinceMonth                      9
CompetitionOpenSinceYear                    2008
Promo2                                         0
Promo2SinceWeek                              NaN
Promo2SinceYear                              NaN
PromoInterval                                NaN
day                                           13
dayofweek                                      6
Name: 4, dtype: object

In [146]:
d1 = train.loc[4,'Date']
d2 = train.loc[423245,'Date']
print d1,d2

2015-07-31 00:00:00 2014-06-17 00:00:00


In [127]:
dd = test.loc[4,'Date']

In [143]:
train.dtypes

Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday             object
SchoolHoliday             int64
dtype: object

In [128]:
print type(dd)

<class 'pandas.tslib.Timestamp'>


In [147]:
(d2-d1).days

-409

In [150]:
(d1-d2).days.weeks

AttributeError: 'int' object has no attribute 'weeks'

In [156]:
test['month'] = test.Date.apply(lambda x: x.month)
test['year'] = test.Date.apply(lambda x: x.year)
test['CompetitionTime'] = test.month - test.CompetitionOpenSinceMonth + (test.year - test.CompetitionOpenSinceYear) * 12

In [157]:
test.head()


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,day,dayofweek,month,
0,1,1,4,2015-09-17,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,17,3,9,...
1,857,1,3,2015-09-16,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,16,2,9,...
2,1713,1,2,2015-09-15,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,15,1,9,...
3,2569,1,1,2015-09-14,1,1,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,14,0,9,...
4,3425,1,7,2015-09-13,0,0,0,0,c,a,1270,9,2008,0,NaN,NaN,NaN,13,6,9,...


In [158]:
test.isnull().sum()

Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                             0
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment                       0
CompetitionDistance             96
CompetitionOpenSinceMonth    15216
CompetitionOpenSinceYear     15216
Promo2                           0
Promo2SinceWeek              17232
Promo2SinceYear              17232
PromoInterval                17232
day                              0
dayofweek                        0
month                            0
year                             0
CompetitionTime              15216
dtype: int64

In [161]:
import datetime
t1 = datetime.datetime.now()
t2 = datetime.datetime.time(datetime.datetime.now())
print type(t1), type(t2)

<type 'datetime.datetime'> <type 'datetime.time'>


In [163]:
print t1
print t2

2015-12-04 17:38:44.437593
17:38:44.437690


In [166]:
print type(t1.day)

<type 'int'>


In [167]:
print t1.hour
print t2.minute

17
38


In [ ]:
print 